In [0]:
sc

Python interpreter will be restarted.
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-832f3249-4bcd-4a0b-83dc-c177079be2f7
    Can't uninstall 'typing-extensions'. No files were found to uninstall.
Python interpreter will be restarted.


In [0]:


# Azure Cosmos DB configuration
cosmos_endpoint = "https://cosmos-df.documents.azure.com:443/"
cosmos_database = "cosmosdb_id"
cosmos_collection = "Cosmoscontainer1"
cosmos_master_key = "z25RNuiX5kWSSs4TDdAa5OwhB06GRnGedPyuRDtieT7Eu0Jb3e2TsbigHEas3A9iCckgyLP1hNLeACDbFTm4oQ=="

from pyspark.sql import SparkSession
 
spark = SparkSession.builder.appName("CosmosDBToDatabricks").getOrCreate()
 
cars_df = spark.read.format("cosmos.oltp") \
    .option("spark.cosmos.accountEndpoint", cosmos_endpoint) \
    .option("spark.cosmos.accountKey", cosmos_master_key) \
    .option("spark.cosmos.database", cosmos_database) \
    .option("spark.cosmos.container", cosmos_collection) \
    .load()
 
cars_df.show()



+----------+---------+----------+-----+------+----------+------+---------+------+--------------------+--------------------+-------+--------+----------+-----------+------+
|DriveTrain|Cylinders|Horsepower| MSRP|Length|      Make|Origin|Wheelbase|Weight|               Model|                  id|Invoice|MPG_City|EngineSize|MPG_Highway|  Type|
+----------+---------+----------+-----+------+----------+------+---------+------+--------------------+--------------------+-------+--------+----------+-----------+------+
|     Front|        6|       185|21595|   182|   Pontiac|   USA|      108|  3779|              Aztekt|a603408f-b705-462...|  19810|      19|       3.4|         26|   SUV|
|       All|        4|       160|19860|   179|     Honda|  Asia|      103|  3258|             CR-V LX|fe2dcdd5-babf-461...|  18419|      21|       2.4|         25|   SUV|
|     Front|        8|       270|39235|   185|Volkswagen|Europe|      106|  3953| Passat W8 4MOTIO...|2e9766fc-b730-471...|  36052|      18|     

In [0]:
cars_df.printSchema()

root
 |-- DriveTrain: string (nullable = true)
 |-- Cylinders: string (nullable = true)
 |-- Horsepower: string (nullable = true)
 |-- MSRP: string (nullable = true)
 |-- Length: string (nullable = true)
 |-- Make: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Wheelbase: string (nullable = true)
 |-- Weight: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- id: string (nullable = false)
 |-- Invoice: string (nullable = true)
 |-- MPG_City: string (nullable = true)
 |-- EngineSize: string (nullable = true)
 |-- MPG_Highway: string (nullable = true)
 |-- Type: string (nullable = true)



In [0]:

from pyspark.sql.functions import *

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Cars Data Processing") \
    .getOrCreate()

In [0]:
# 1. Remove specified columns
columns_to_remove = ["_attachments", "_etag", "_rid", "_self", "_ts", "id"]
cars_df = cars_df.drop(*columns_to_remove)
cars_df.printSchema()

root
 |-- DriveTrain: string (nullable = true)
 |-- Cylinders: string (nullable = true)
 |-- Horsepower: string (nullable = true)
 |-- MSRP: string (nullable = true)
 |-- Length: string (nullable = true)
 |-- Make: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Wheelbase: string (nullable = true)
 |-- Weight: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- Invoice: string (nullable = true)
 |-- MPG_City: string (nullable = true)
 |-- EngineSize: string (nullable = true)
 |-- MPG_Highway: string (nullable = true)
 |-- Type: string (nullable = true)



In [0]:
# Convert required numerical columns to numerical types
numerical_columns = ["EngineSize", "Horsepower", "Invoice", "Length", "MPG_City", "MPG_Highway", "MSRP", "Weight", "Wheelbase"]

for column in numerical_columns:
    cars_df = cars_df.withColumn(column, col(column).cast("float"))  # or use "int" if the column should be integer

# Show DataFrame after conversion
cars_df.printSchema()  # Check the schema to verify changes

root
 |-- DriveTrain: string (nullable = true)
 |-- Cylinders: string (nullable = true)
 |-- Horsepower: float (nullable = true)
 |-- MSRP: float (nullable = true)
 |-- Length: float (nullable = true)
 |-- Make: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Wheelbase: float (nullable = true)
 |-- Weight: float (nullable = true)
 |-- Model: string (nullable = true)
 |-- Invoice: float (nullable = true)
 |-- MPG_City: float (nullable = true)
 |-- EngineSize: float (nullable = true)
 |-- MPG_Highway: float (nullable = true)
 |-- Type: string (nullable = true)



In [0]:
# Filter cars with Horsepower greater than 200
high_hp_cars = cars_df.filter(col("Horsepower") > 200).select("Make", "Model", "MSRP")
high_hp_cars.show(5)


+----------+--------------------+-------+
|      Make|               Model|   MSRP|
+----------+--------------------+-------+
|Volkswagen| Passat W8 4MOTIO...|39235.0|
|Volkswagen|         Phaeton 4dr|65000.0|
|       GMC|      Sierra HD 2500|29322.0|
|     Acura|          3.5 RL 4dr|43755.0|
|Mitsubishi| Eclipse Spyder G...|26992.0|
+----------+--------------------+-------+
only showing top 5 rows



In [0]:
# Filter out rows with null values in 'Horsepower' or 'MPG_City'
cleaned_data = cars_df.filter(col("Horsepower").isNotNull() & col("MPG_City").isNotNull())
cleaned_data.select("Make", "Model", "Horsepower","MPG_City").show()


+----------+--------------------+----------+--------+
|      Make|               Model|Horsepower|MPG_City|
+----------+--------------------+----------+--------+
|   Pontiac|              Aztekt|     185.0|    19.0|
|     Honda|             CR-V LX|     160.0|    21.0|
|Volkswagen| Passat W8 4MOTIO...|     270.0|    18.0|
|Oldsmobile|       Alero GLS 2dr|     170.0|    20.0|
|     Mazda| B2300 SX Regular...|     143.0|    24.0|
|Volkswagen|         Phaeton 4dr|     335.0|    16.0|
|       GMC|      Sierra HD 2500|     300.0|    13.0|
|   Hyundai| Elantra GT 4dr h...|     138.0|    26.0|
|     Acura|          3.5 RL 4dr|     225.0|    18.0|
|Mitsubishi| Eclipse Spyder G...|     210.0|    21.0|
|    Toyota|           Matrix XR|     130.0|    29.0|
|   Lincoln|    LS V6 Luxury 4dr|     232.0|    20.0|
|     Dodge|   Grand Caravan SXT|     215.0|    18.0|
|    Toyota|       Highlander V6|     230.0|    18.0|
|       GMC|          Safari SLE|     190.0|    16.0|
|      Ford|       Focus ZX5

In [0]:
# Calculate average MSRP and max Horsepower by Type
agg_metrics = cars_df.groupBy("Type").agg(
    {"MSRP": "avg", "Horsepower": "max"}
).withColumnRenamed("avg(MSRP)", "Average_MSRP") \
 .withColumnRenamed("max(Horsepower)", "Max_Horsepower")
agg_metrics.show()

+------+------------------+--------------+
|  Type|      Average_MSRP|Max_Horsepower|
+------+------------------+--------------+
|Sports|53793.145833333336|         500.0|
|   SUV| 34447.41071428572|         340.0|
| Sedan| 29715.78947368421|         493.0|
|Hybrid|           20325.0|         110.0|
| Truck| 22967.18181818182|         345.0|
| Wagon|          29188.25|         340.0|
+------+------------------+--------------+



In [0]:
cars_df = cars_df.withColumn("priceCategory", 
                   when(col("MSRP") > 50000, "costly")
                   .when(col("MSRP") < 25000, "cheaper")
                   .otherwise("average")) 

In [0]:
cars_df.printSchema()

root
 |-- DriveTrain: string (nullable = true)
 |-- Cylinders: string (nullable = true)
 |-- Horsepower: float (nullable = true)
 |-- MSRP: float (nullable = true)
 |-- Length: float (nullable = true)
 |-- Make: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Wheelbase: float (nullable = true)
 |-- Weight: float (nullable = true)
 |-- Model: string (nullable = true)
 |-- Invoice: float (nullable = true)
 |-- MPG_City: float (nullable = true)
 |-- EngineSize: float (nullable = true)
 |-- MPG_Highway: float (nullable = true)
 |-- Type: string (nullable = true)
 |-- priceCategory: string (nullable = false)

